In [51]:
import math
from collections import namedtuple


class Stack:
    def __init__(self):
        self.top = None
        self.size = 0

    def __len__(self):
        return self.size

    def __iter__(self):
        return self.Iter(self.top)

    class Iter:
        def __init__(self, top):
            self.current = top

        def __iter__(self):
            return self

        def __next__(self):
            if self.current == None:
                raise StopIteration
            result = self.current.value
            self.current = self.current.next
            return result

    class NodeStack:
        def __init__(self, value):
            self.value = value
            self.next = None

    def __repr__(self):
        represent = ''
        current = self.top
        while current:
            represent += f'{current.value} --> '
            current = current.next
        return represent

    def push(self, value):
        new_node = self.NodeStack(value)
        if not self.size:
            self.top = new_node
        else:
            new_node.next = self.top
            self.top = new_node
        self.size += 1

    def pop(self):
        if not self.size:
            return
        popped_node = self.top
        self.top = self.top.next
        self.size -= 1
        return popped_node.value

    def __contains__(self, value):
        current = self.top
        while current:
            if current.value == value:
                return True
            current = current.next
        return False


# Island count


grid = [
    ['W', 'L', 'W', 'W', 'W'],
    ['W', 'L', 'W', 'W', 'W'],
    ['W', 'W', 'W', 'L', 'W'],
    ['W', 'W', 'L', 'L', 'W'],
    ['L', 'W', 'W', 'L', 'L'],
    ['L', 'L', 'W', 'W', 'W'],
]


Position = namedtuple('Position', ('row', 'column'))


def island_count(grid):
    count = 0
    stack_bk = Stack()
    for row in range(len(grid)):
        for column in range(len(grid[row])):
            position = Position(row, column)
            if position not in stack_bk:
                stack_bk.push(position)
                if grid[position.row][position.column] == 'L':
                    explore(grid, position, stack_bk)
                    count +=1
    return count


def explore(grid, position, stack_bk):
    stack = Stack()
    stack.push(position)
    while stack.size:
        popped = stack.pop()
        row, column = popped.row, popped.column
        neighbors = (
            Position(row - 1, column), Position(row + 1, column),
            Position(row, column + 1), Position(row, column - 1)
        )
        for neighbor in neighbors:
            if neighbor not in stack_bk:
                try:
                    if not (0 <= neighbor.row <= len(grid)) \
                            or not (0 <= neighbor.column <= len(grid[0])):
                        raise IndexError
                    if grid[neighbor.row][neighbor.column] == 'L':
                        stack.push(neighbor)
                    stack_bk.push(neighbor)
                except IndexError:
                    continue


def island_count_recursive(grid):
    stack_bk = Stack()
    grid_cells = (Position(row, column)
                  for row in range(len(grid))
                  for column in range(len(grid[row])))
    def inner():
        try:
            position = next(grid_cells)
            count = 0
            if position not in stack_bk:
                cell = grid[position.row][position.column]
                if cell == 'W':
                    stack_bk.push(position)
                if cell == 'L':
                    explore_recursive(grid, position, stack_bk)
                    count = 1
            return count + inner()
        except StopIteration:
            return 0
    return inner()


def explore_recursive(grid, position, stack_bk):
    def inner(position):
        try:
            row, column = position.row, position.column
            if not (0 <= row <= len(grid)) \
                or not (0 <= column <= len(grid[0])):
                raise IndexError
            cell = grid[row][column]
            if position not in stack_bk:
                stack_bk.push(position)
                if cell == 'L':
                    for neighbor in (
                        Position(row - 1, column), Position(row + 1, column),
                        Position(row, column + 1), Position(row, column - 1)
                    ):
                        inner(neighbor)
        except IndexError:
            return
    inner(position)


# Minimum island


grid = [
    ['W', 'L', 'W', 'W', 'W'],
    ['W', 'L', 'W', 'W', 'W'],
    ['W', 'W', 'W', 'L', 'W'],
    ['W', 'W', 'L', 'L', 'W'],
    ['L', 'W', 'W', 'L', 'L'],
    ['L', 'L', 'W', 'W', 'W'],
]


Position = namedtuple('Position', ('row', 'column'))


def minimum_island(grid):
    stack_bk = Stack()
    minimum = math.inf
    for row in range(len(grid)):
        for column in range(len(grid[row])):
            position = Position(row, column)
            if position not in stack_bk:
                stack_bk.push(position)
                if grid[position.row][position.column] == 'L':
                    min_size = explore_size(grid, position, stack_bk)
                    if min_size < minimum:
                        minimum = min_size
    return minimum


def explore_size(grid, position, stack_bk):
    size = 0
    stack = Stack()
    stack.push(position)
    while stack.size:
        popped = stack.pop()
        size += 1
        row, column = popped.row, popped.column
        for neighbor in (
            Position(row - 1, column), Position(row + 1, column),
            Position(row, column + 1), Position(row, column - 1)
        ):
            if neighbor not in stack_bk:
                try:
                    if not (0 <= neighbor.row <= len(grid)) \
                        or not (0 <= neighbor.column <= len(grid[0])):
                        raise IndexError
                    if grid[neighbor.row][neighbor.column] == 'L':
                        stack.push(neighbor)
                    stack_bk.push(neighbor)
                except IndexError:
                    continue
    return size


def minimum_island_recursive(grid):
    stack_bk = Stack()
    grid_cells = (Position(row, column) for row in range(len(grid))
                    for column in range(len(grid[row])))
    def inner():
        try:
            position = next(grid_cells)
            size = math.inf
            if position not in stack_bk:
                cell = grid[position.row][position.column]
                if cell == 'W':
                    stack_bk.push(position)
                if cell == 'L':
                    size = explore_size_recursive(grid, position, stack_bk)
            return min(size, inner())
        except StopIteration:
            return math.inf
    return inner()


def explore_size_recursive(grid, position, stack_bk):
    def inner(position):
        try:
            row, column = position.row, position.column
            if not (0 <= row <= len(grid)) \
                    or not (0 <= column <= len(grid[0])):
                raise IndexError
            if position in stack_bk:
                return 0
            stack_bk.push(position)
            if grid[row][column] == 'W':
                return 0
            return 1 + sum(inner(neighbor) for neighbor in (
                Position(row - 1, column), Position(row + 1, column),
                Position(row, column + 1), Position(row, column - 1)
            ))
        except IndexError:
            return 0
    return inner(position)
